Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1,  image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size ** 2, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    # Set small learning rate 
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    # train_prediction = tf.nn.softmax(logits) # logits is equal to `tf.matmul(tf_train_dataset, weights) + biases`
    train_prediction = tf.nn.softmax(tf.matmul(tf_train_dataset, weights) + biases)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [5]:
num_steps = 801

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, and random.
    # tf.global_variables_initializer().run()
    tf.initialize_all_variables().run() 
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: 21.284416
Training accuracy: 4.9%
Validation accuracy: 6.3%
Loss at step 100: 2.285535
Training accuracy: 72.1%
Validation accuracy: 70.6%
Loss at step 200: 1.820697
Training accuracy: 75.3%
Validation accuracy: 72.9%
Loss at step 300: 1.574235
Training accuracy: 76.6%
Validation accuracy: 73.7%
Loss at step 400: 1.411900
Training accuracy: 77.5%
Validation accuracy: 74.1%
Loss at step 500: 1.292410
Training accuracy: 78.2%
Validation accuracy: 74.5%
Loss at step 600: 1.199489
Training accuracy: 78.8%
Validation accuracy: 74.7%
Loss at step 700: 1.124398
Training accuracy: 79.4%
Validation accuracy: 74.8%
Loss at step 800: 1.061913
Training accuracy: 79.9%
Validation accuracy: 74.9%
Test accuracy: 82.8%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size ** 2, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    # tf.global_variables_initializer().run()
    tf.initialize_all_variables().run() 
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 15.597815
Minibatch accuracy: 3.9%
Validation accuracy: 9.6%
Minibatch loss at step 500: 1.670545
Minibatch accuracy: 75.8%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 1.472620
Minibatch accuracy: 75.8%
Validation accuracy: 76.8%
Minibatch loss at step 1500: 1.002543
Minibatch accuracy: 80.5%
Validation accuracy: 77.2%
Minibatch loss at step 2000: 0.858863
Minibatch accuracy: 80.5%
Validation accuracy: 77.3%
Minibatch loss at step 2500: 0.811599
Minibatch accuracy: 80.5%
Validation accuracy: 78.6%
Minibatch loss at step 3000: 0.759991
Minibatch accuracy: 82.8%
Validation accuracy: 78.4%
Test accuracy: 85.0%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [8]:
# ReLU neuron

# param
learning_rate = 0.001
training_epochs = 30
batch_size = 521
display_step = 1

# hyper-parameter
n_hidden_1 = 256 
n_input = 784 # img shape: 28*28
n_classes = 10 # MNIST total classes (0-9 digits)

graph = tf.Graph()
with graph.as_default():
    # init weights
    weights_hiden =  tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=np.sqrt(n_input)))
    weights_out = tf.Variable(tf.random_normal([n_hidden_1, n_classes], stddev=np.sqrt(n_hidden_1)))
    
    biases_hidden = tf.Variable(tf.random_normal([n_hidden_1]))
    biases_out = tf.Variable(tf.random_normal([n_classes]))
    
    x = tf.placeholder("float", [None, n_input])
    y = tf.placeholder("float", [None, n_classes])
    
    def model(x, weights_hiden, weights_out, biases_hidden, biases_out):
        # hidden layer with RELU activation
        layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights_hiden), biases_hidden))
        # output layer with linear activation
        out_layer = tf.matmul(layer_1, weights_out) + biases_out
        return out_layer
    
    # Construct model
    pred = model(x, weights_hiden, weights_out, biases_hidden, biases_out)
    
    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    

In [9]:
# Launch the graph
with tf.Session(graph=graph) as sess:
    tf.initialize_all_variables().run()
    print('Initialized')
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(train_dataset.shape[0]/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x = train_dataset[(i*batch_size):((i*batch_size) + batch_size), :]
            batch_y = train_labels[(i*batch_size):((i*batch_size) + batch_size), :]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Test data accuracy:", accuracy.eval({x: test_dataset, y: test_labels}))
    print("Valid data accuracy:", accuracy.eval({x: valid_dataset, y: valid_labels}))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Epoch: 0001 cost= 60023.653016890
Epoch: 0002 cost= 32639.976562500
Epoch: 0003 cost= 23565.862541816
Epoch: 0004 cost= 19611.291525579
Epoch: 0005 cost= 17346.721248776
Epoch: 0006 cost= 15822.979260158
Epoch: 0007 cost= 14689.134872715
Epoch: 0008 cost= 13784.768411900
Epoch: 0009 cost= 13036.001053056
Epoch: 0010 cost= 12400.773725624
Epoch: 0011 cost= 11853.488128264
Epoch: 0012 cost= 11373.727987822
Epoch: 0013 cost= 10943.847623103
Epoch: 0014 cost= 10557.838728225
Epoch: 0015 cost= 10207.821396153
Epoch: 0016 cost= 9889.296015727
Epoch: 0017 cost= 9598.624136902
Epoch: 0018 cost= 9331.297105754
Epoch: 0019 cost= 9084.141368258
Epoch: 0020 cost= 8855.413214701
Epoch: 0021 cost= 8642.657753090
Epoch: 0022 cost= 8443.966498552
Epoch: 0023 cost= 8257.138492116
Epoch: 0024 cost= 8081.087936521
Epoch: 0025 cost= 7914.670564417
Epoch: 0026 cost= 7757.235656260
Epoch: 0027 cost= 7607.572157260
Epoch: 0